- [ ] gets stuck at the main process not aving local energy metrics
- [ ] use decorator or helper function to use all_reduce + barrier + logging + timeout handler correctly

- [ ] move the run_experiment() into its own dedicated script so it can use accelerate.launch and make use of Terminate_on_errors=True
    - [ ] add in try and except clauses and make it robsut to failovers

- [ ] work on grid search experiment setup: 
    - [ ] implement the accelerate launcher (as script) that terminate_on_error=True -> experiment continues and restarts even if it hits an error
    - [ ] a grid search through the different variables
    - [ ] implement various try an except clauses so that it is robust to keep running and restarting the kernel if necessary
    - [ ] get it to wait if it gets stuck then retry again...
    - [ ] set up warm up period with dummy content to warm up the cores -> discard initial
    - [ ] set up repetition and interleaving of experiment types (see literature)


- [ ] check all config types work
    - [ ] everything breaks FLOPs (if decoder temp >1  / if input length is longer... many things)
    - [ ] doesn't work on more than 2 GPUs
    - [ ] adaptive batching doesn't work
    - [ ] sort out sharding configs

  
- [ ] set up chain of reasoning workflow
- [ ] do data work: what kind of data makes a difference
  - [ ] task complexity

- [ ] in the inference function i am generating raw_text_outputs which get assigned to a variable then discarded... this could be a lot of memory: add an ExperimentalConfig

- [ ] the text outputs seem determinisitc...

---


- [ ] decoder temp: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
    - change this: so that there's the option in the experimental_configs to chose gredy decoding, beam search, random sampling, top-k sampling, top-p sampling
        - params: do_sample; temperature, top_p, top_k
        - Greedy deocing?
        - Beam Search
        - random sampling (basic): outputs = model.generate(input_ids, do_sample=True)
        - Top-k Sampling (Controlled Randomness): outputs = model.generate(input_ids, do_sample=True, top_k=50)
        -  Top-p (Nucleus) Sampling (Dynamic Control): outputs = model.generate(input_ids, do_sample=True, top_p=0.9)

- [ ] set up AMD CPU logic (currently only INTEL)
- [ ] take the GPU & CPU utilisation metrics within the infernece workflow (currently they are done after all inferences are complete) -> one measurement per batch -> avg over them all for the process

# Low priority
- [ ] `is_encoder_decoder` shows "false" it should be one or the other
- [ ] I don't think decoder temp does anything (I don't think it really matters)
- [ ] add Carbon Tracker
- [ ] add power over time to the metrics
- [ ] Measure L2 cash + stalls while system is waiting 
- [ ] add logic for task-specific metrics in the reporting
- [ ] make experimental configs enterable as a YAML file rather than directly
- [ ] Add in warm up period before code carbon measurement+ inference run... Also at experiment level: interleave 

# Done
- [x] change codeCarbon log_level to warning 
- [x] Code Carbon turn off multiple runs in allow_multiple_runs
- [x] calculate computation metrics
- [x] put the text_gen_inference() into the helper functionn (as it is pretty generic)
- [x] us Optimum library 
- [x] sort out distributed inference using accelerate / vLLM. At the moment it's not working on either
- [x] aggregate process metrics 
    - once it goes to multi-GPU it gets weird - i think this is why half the runs don't work, the other half do
    - including assinging the unique ID
    - decide which metrics to sum, which to avg
- [x] check why the CodeCarbon CSV metrics and the JSON metrics don't align - Answer: because they are per process -> allows me to validate (also check that CodeCarbon don't offer their own aggregation)
    - CC can do by machine or by process
        - parameter: tracking_mode -> machine measure the power consumptions of the entire machine (defaults) / process try and isolate the tracked processes in isolation
- [x] refactor: (i) helper functions; (ii) Object oriented
- [x] Change experimental_vars list:
    - [x] move the GPU list up to the experiment_setup JSON object (below counts of GPUs)... also get it to validate against which GPUs it actually ran on
    - [x] experiment_setup: move the GPU count and type above the CPU count and type
- [x] + also add if model in encoder-decoder Vs decoder only model, to see if these behave differently under different optimisations  (save that under experiment set up)
- [x] move some of the experiment utils to metrics,py 
    - [x] rename metrics_results.py
    - [x] rename function `aggregate_experiments` to `aggregate_experiment_results`
- [x] work out what the GPU and CPU power metrics coming out of CodeCarbon are referring to (avg power draw?)
- [x] GPU & CPU power = 'avg across processors' <- add this in (ram power constant)
- [x] GPU & CPU energy = sum across processort
- [x] NB the RAM energy seems to be wrong (I am summing, but RAM is shared ????)
- [x] GPU list: doesn't run on different GPUs (somewhere it's hard coded to 01)
- [x] review compute metrics:
    - [x] remove cpu vendor 
    - [x] remove cuda 
- [x] sort out the print statements in experiments to supress (i) the code carbon output, (ii) the final JSON print
- [x] distributed env issues
- [x] Add in tunable optimisations into experimental variables: floating points, quantisation, batching, sharding
- [x] currently the infernece and compute metrics are taken from main process, but they should probably be taken acorss all processes and averaged
- [x] "number_input_prompts": null doesn;t make sense
- [x] "used_gpu": "cuda:0" doesn't make sense
- [x] the processes are not aggregating into a single experiment (the base experiment is there, but it also enumerates other processes and prints them
- [x] clean up (remove) "experiment_setup": {}, and  "experiment_variables": {},
- [x] sort out why the tokens are so many : "Token indices sequence length is longer than the specified maximum sequence length for this model (7921 > 2048). Running this sequence through the model will result in indexing errors"
- [x] its not counting FLOPs --> change it to calculate FLOPS over whole dataset, NOT JUST SINGLE SAMPLE
- [x] FLOPs is currently summed in the aggregation step, but i think it should be taken as a constant across all processes?
- [x] review compute metrics:
    - [x] sort out memory allocation metrics (they repeat, are they ever different?)
    - [x] sort out the gpu_utilisation_percent <- currently it shows overall utilisation by other programmes, but I think that is right? check? [now it seems to be set to always 0... whereas when Silke was running training it was much more variable, did I break something or is notone using the GPUs currently?]
- [x] FLOPs monitoring
- [x] to experimental_variables: add avg_input_tokens and avg_output_tokens between and within batches / across the overall experiment??
- [x] log metrics both at (1) the individual per prompt, (2) the per-batch level, and (3) the experiment level (e.g. number of input/output tokens, energy etc etc)
- [x] IT DOESN'T WORK ON MORE THAN 2 GPUS
- [x] make some changes between ExperimentConfig and ExperimentRunner
    - [x] take the "inference_fn" of the notebook_launcher and instead have it determined in the run_experiment flow, by the parameter "purely_generative" of the experiment_config object.
- [x] from run_gen_inf() remove line `do_sample = True if inference_type == "reasoning" else False`
- [x] how is final GPU / CPU power calculated by CodeCarbon? is it an avg?
- [x] listen to Lynn's call --> decide research question and begin writing it up
- [x] sort out aggregate_results method to save into a single JSON file / dataframe
    - [x] get this to calculate token efficiency!!!
- [x] decode token ids doesn't work